# Total script

In [13]:
def linkedIn_Scraper():
    #importing drivers and selenium
    from selenium import webdriver
#     from selenium.webdriver.support.ui import Select
#     from selenium.webdriver.support.ui import WebDriverWait 
#     from selenium.webdriver.chrome.options import Options
#     from selenium.common.exceptions import ElementClickInterceptedException
#     from selenium.webdriver.common.action_chains import ActionChains
#     from selenium.webdriver.common.keys import Keys
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    from string import digits
    from difflib import get_close_matches 
    import random
    import time
    import re
    import firebase_admin
    from firebase_admin import credentials
    from firebase_admin import firestore
    from firebase_admin import db

    start_time = time.time()
    sleep_time_0 = random.randrange(5000,10999)/10000
    sleep_time_1 = random.randrange(4000,6999)/10000

    # ----- Getting the links ----------#

    #chrome_options = Options()
    #chrome_options.add_argument("--headless")
    driver = webdriver.Chrome()

    driver.get("https://www.linkedin.com/jobs/data-scientist-jobs?currentJobId=1988654269&position=1&pageNum=0")
    time.sleep(sleep_time_1)
    while not driver.find_elements_by_xpath('/html/body/header/section/form/ul/li[2]/div/button'):
        driver.delete_all_cookies()
        driver.quit()        
        #chrome_options = Options()
        #chrome_options.add_argument("--headless")
        driver = webdriver.Chrome()
        driver.get("https://www.linkedin.com/jobs/data-scientist-jobs?currentJobId=1988654269&position=1&pageNum=0")

    #setting fiolter to get latest job
    driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/button').click()
    driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/div/fieldset/div[1]/ul/li[1]/label').click()
    driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/div/fieldset/div[2]/button').click()

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while i<200:
        i=i+1
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(sleep_time_1)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    for j in range(1,5):
        if driver.find_elements_by_xpath('//button[text()="See more jobs"]'):
            element=driver.find_element_by_xpath('//button[text()="See more jobs"]')
            driver.execute_script("arguments[0].click();", element)
            time.sleep(random.randrange(1000,2999)/10000)
            j=j+1 
        else:
            break

    links = driver.find_elements_by_xpath("//a[@class='result-card__full-card-link']")
    links_list =[link.get_attribute('href') for link in links]

    print("length of links: "+ str(len(links_list)))

    driver.delete_all_cookies()
    driver.quit()
    duration1=time.time() - start_time
    print("Getting links time: "+str(duration1))

    # ----- Getting the data from links ----------#

    #chrome_options = Options()
    #chrome_options.add_argument("--headless")
    driver = webdriver.Chrome()
    outer_dict={}
    for link in links_list:
        link_key = link.split('?')[0]
        driver.get(link)
        time.sleep(sleep_time_1)

        try:
            job_title = driver.find_element_by_class_name('topcard__title').text 
        except:
            driver.delete_all_cookies()
            continue

        try:
            company_name =driver.find_element_by_xpath('/html/body/main/section[1]/section[2]/div[1]/div[1]/h3[1]/span[1]/a').text
        except:
            driver.delete_all_cookies()
            continue
        try:
            location = driver.find_element_by_xpath('/html/body/main/section[1]/section[2]/div[1]/div[1]/h3[1]/span[2]').text
        except:
            pass

        try:
            element=driver.find_element_by_partial_link_text('Show more')
            driver.execute_script("arguments[0].click();", element)
        except:
            pass

        job_description = driver.find_element_by_class_name('show-more-less-html__markup').get_attribute('innerHTML')
        cleaner = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        job_description = re.sub(cleaner, ' ', job_description)
        if 'â€™' in job_description:
            job_description.replace('â€™', "'")
        headers = driver.find_elements_by_class_name('job-criteria__subheader')
        infos =driver.find_elements_by_class_name('job-criteria__item')
        remote=0
        if 'remote' in job_title.lower() or 'remote' in job_description.lower() or 'remote' in location.lower():
            remote=1
        
        inner_dict={
            'about':{
            'title':job_title,
            'company':company_name,
            'location':location,
            'url' :link
            },
            'jobDescription':job_description,
            'linkKey':link_key,
            'linkedIn': True,
            'remote':remote
        }
        for i,info in enumerate(infos):
            h=headers[i].text
            infoVal=re.sub(cleaner, ',', info.get_attribute('innerHTML')).strip()
            x=infoVal.split(h)[1].strip()
            y=x[2:-1]
            l=y.split(',,')
            inner_dict['about'][h]=','.join(l)

            driver.delete_all_cookies()
            outer_dict[link_key]=inner_dict
        print(len(outer_dict))

    duration2=time.time() - start_time
    print("Getting data time: "+str(duration2))
    print("length of outer dicty: "+str(len(outer_dict)))
    driver.delete_all_cookies()
    driver.quit()

    # ----- Storing the data in firebase ----------#

    mycred = credentials.Certificate('firebase-key.json')

    try:
        firebase_admin.initialize_app(mycred)
    except ValueError:
        pass

    db = firestore.client()
    all_docs = db.collection('jobs').where(u'linkedIn', u'==', True).stream()

    linkKeyList=[doc.to_dict()['linkKey'] for doc in all_docs]

    keys =list(outer_dict.keys())
    for linkKey in keys:
        if linkKey not in linkKeyList:
            doc_ref = db.collection('jobs').document()
            doc_ref.set(outer_dict.get(linkKey))

    finalDuration = time.time()-start_time
    print("final time:"+ str(finalDuration))

In [14]:
linkedIn_Scraper()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 84


# First half of the scraper that uses GCP storage

In [13]:


import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from string import digits
from difflib import get_close_matches 
import json
import random
import time
import re

start_time = time.time()
sleep_time_1 = random.randrange(4000,6999)/10000

# ----- Getting the links ----------#

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://www.linkedin.com/jobs/data-scientist-jobs?currentJobId=1988654269&position=1&pageNum=0")
time.sleep(sleep_time_1)
while not driver.find_elements_by_xpath('/html/body/header/section/form/ul/li[2]/div/button'):
    driver.delete_all_cookies()
    driver.quit()        
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.linkedin.com/jobs/data-scientist-jobs?currentJobId=1988654269&position=1&pageNum=0")

#setting fiolter to get latest job
driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/button').click()
driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/div/fieldset/div[1]/ul/li[1]/label').click()
driver.find_element_by_xpath('/html/body/header/section/form/ul/li[2]/div/div/fieldset/div[2]/button').click()

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
i=0
while i<200:
    i=i+1
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(sleep_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

for j in range(1,5):
    if driver.find_elements_by_xpath('//button[text()="See more jobs"]'):
        element=driver.find_element_by_xpath('//button[text()="See more jobs"]')
        driver.execute_script("arguments[0].click();", element)
        time.sleep(random.randrange(1000,2999)/10000)
        j=j+1 
    else:
        break

links = driver.find_elements_by_xpath("//a[@class='result-card__full-card-link']")
links_list =[link.get_attribute('href') for link in links]
links_chunks = [links_list[x:x+100] for x in range(0, len(links_list), 100)]

print("length of links: "+ str(len(links_list)))

jobs = {
    'links':links_chunks
}

with open('links1.json', 'w') as json_file:
    json.dump(jobs, json_file)

from google.cloud import storage
# Setting credentials using the downloaded JSON file

client = storage.Client.from_service_account_json(json_credentials_path='dataprime-dev-app-default-service-account.json')

# Creating bucket object

bucket = client.get_bucket('linkedin_job_posting_links')

# Name of the object to be stored in the bucket

blob = bucket.blob('links2.json')

# Name of the object in local file system

blob.upload_from_filename('links2.json')



driver.delete_all_cookies()
driver.quit()
duration1=time.time() - start_time
print("Getting links time: "+str(duration1))


length of links: 174
Getting links time: 15.791212558746338
